##### Copyright 2022 The Cirq Developers

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# QVM Basic Example

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/simulate/qvm_basic_example"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/simulate/qvm_basic_example.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/simulate/qvm_basic_example.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/simulate/qvm_basic_example.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

This notebook walks through running a simple circuit on the [Quantum Virtual Machine](/cirq/simulate/quantum_virtual_machine), including the necessary constraints on a device-runnable circuit and how to satisfy them. 

## **Install** Cirq and qsim
Note: this notebook relies on unreleased Cirq features. If you want to try these features, make sure you install Cirq via `pip install cirq --pre`.

In [ ]:
# @title Install `cirq_google` and `qsimcirq`

try:
    import cirq
    import cirq_google
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq --pre cirq-google
    print("installed cirq.")
    import cirq
    import cirq_google

try:
    import qsimcirq
except ImportError:
    print("installing qsimcirq...")
    !pip install --quiet qsimcirq
    print(f"installed qsimcirq.")
    import qsimcirq

# Other modules used in this colab
import matplotlib.pyplot as plt
import time

## Create a **Quantum Virtual Machine**

The following cell builds a Quantum Virtual Machine that mimics a particular Google quantum hardware device (currently Rainbow or Weber) using the following customizable steps: 
- Constructing a `cirq.NoiseModel` object from device calibration data saved in Cirq. See [Representing Noise](/cirq/noise/representing_noise) for more on noise models. 
- Building a `qsimcirq.QsimSimulator` that uses this noise model. See [Noisy Simulation](/cirq/simulate/noisy_simulation) and [Noise simulation with qsim](/qsim/tutorials/noisy_qsimcirq) for more. 
- Creating a `cirq.Device` that imposes the same constraints on circuits that the original device would. See [Devices](/cirq/hardware/devices) for more on these constraint objects. 
- Packaging the simulator and device into an object that implements the `cirq.Engine` interface that the hardware device would use. 

If you don't need this level of control, you can also instantiate a QVM with `cirq_google.engine.create_default_noisy_quantum_virtual_machine`, as in [QVM Creation Template](/cirq/simulate/qvm_builder_code). 

In [ ]:
# @title Choose a processor ("rainbow" or "weber")
processor_id = "weber"  # @param {type:"string"}

# Construct a simulator with a noise model based on the specified processor.
cal = cirq_google.engine.load_median_device_calibration(processor_id)
noise_props = cirq_google.noise_properties_from_calibration(cal)
noise_model = cirq_google.NoiseModelFromGoogleNoiseProperties(noise_props)
sim = qsimcirq.QSimSimulator(noise=noise_model)

# Create a device from the public device description
device = cirq_google.engine.create_device_from_processor_id(processor_id)
# Build the simulated local processor from the simulator and device.
sim_processor = cirq_google.engine.SimulatedLocalProcessor(
    processor_id=processor_id, sampler=sim, device=device, calibrations={cal.timestamp // 1000: cal}
)
# Package the processor to use an Engine interface
sim_engine = cirq_google.engine.SimulatedLocalEngine([sim_processor])
print(
    "Your quantum virtual machine",
    processor_id,
    "is ready, here is the qubit grid:",
    "\n========================\n",
)
print(sim_engine.get_processor(processor_id).get_device())

## **Create** a circuit and **transform** it to make it executable on the Google quantum processor.

The circuit you use needs to be _device ready_, which means it: 
- Is applied to qubits that exist on the device. 
- Respects the connectivity of qubits on the device.
- Is comprised of operations from the device's gate set. 

Below is an example of how a circuit is prepared to be run on the QVM.

### Create a GHZ state builder circuit

The generalized [Greenberger–Horne–Zeilinger (GHZ) state](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state){:.external} has the form $\frac{|00..0⟩ + |11..1⟩}{\sqrt{2}}$ and, in this case, will be constructed using 10 qubits, with a Hadamard and a sequence of CNOT gates:

In [ ]:
# Define an abstract line of 10 qubits
number_of_qubits = 10
qubits = cirq.LineQubit.range(number_of_qubits)

# Create a GHZ circuit on this qubit line
ghz_circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    *[cirq.CNOT(qubits[0], qubits[i]) for i in range(1, number_of_qubits)],
    cirq.measure(*qubits, key='out'),
)
print(ghz_circuit)

### Fit the circuit to the device with mapping and routing

The circuit's qubits need to be mapped to physical device qubits. Additionally, grid devices don't support two-qubit gates between non-adjacent qubits, meaning that some SWAP gates may need to be added in order to fit the circuit onto the device.

Do this with Cirq's [Routing Transformer](cirq/transform/routing_transformer). Specifically, `cirq.RouteCQC` both selects qubits and adds swaps for you. The cell below routes the GHZ circuit, adding a couple SWAPs in doing so.

In [ ]:
# Prepare the router with the device's graph.
router = cirq.RouteCQC(device.metadata.nx_graph)
# Transform the circuit with the router.
routed_circuit = router(ghz_circuit)
print(routed_circuit)

If you want to select a particular subset of qubits that your circuit will run on, perhaps in order to use more performant qubits, you can take a [subgraph](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.subgraph.html){.:external} of the device's graph and pass that to `cirq.RouteCQC` instead of the whole graph. See [Qubit Picking](/cirq/hardware/qubit_picking) for more on how to select the qubits to use.

Also, it is quite important to note that the routed circuit is unlikely to be unitarily equivalent to the circuit you passed in. This is because, as the qubit states are SWAPped around, they are not necessarily swapped back into their original location for efficiency reasons. However, the measurement operations on your circuit will have the same keys, so you should use those as normal. See [Routing Transformer#Unitary Equivalence](cirq/transform/routing_transformer#unitary_equivalence) for more details.

You can also choose to manually map your circuit to the device's qubits and ensure that your circuit's topology is supported by the device's. This may be useful if you need to squeeze extra performance out of the device by placing circuit qubits with many operations on less error-prone device qubits. This approach is used in the [QVM Stabilizer Example](cirq/simulate/qvm_stabilizer_example) page, which selects qubits manually and transforms the circuit to use them.

### Transform the circuit 

Before executing a circuit on (virtual) quantum hardware, the operations in the circuit need to be translated to use the types of gates the device supports. The `cirq.optimize_for_target_gateset` function does this for you, transforming the operations to use the `cirq.SqrtIswapTargetGateset`, which is supported by the Weber processor that this QVM is based on. Learn more about the gate set constraints of Google hardware at the [Hardware](/hardware) page.

In [ ]:
# Convert the gates in the GHZ circuit to the "SqrtIswap" gateset, which the device uses.
translated_circuit = cirq.optimize_for_target_gateset(
    routed_circuit, context=cirq.TransformerContext(deep=True), gateset=cirq.SqrtIswapTargetGateset()
)
print(translated_circuit)

Finally, ensure that the circuit is runnable on the device by validating it.

In [ ]:
device.validate_circuit(translated_circuit)

### **Running other circuits**

In principle, you can run any custom [Circuit](/cirq/build/circuits) with a quantum virtual machine, but realistically there are some constraints. As mentioned, the circuits need to be mappable to the device. Additionally, the number of qubits that are simulatable depends highly on the hardware available to you and how long you are able to run your simulation. As the QVM is instantiated as above, with a `qsimcirq.QSimSimulator`, it only uses the default, local [qsim](https://quantumai.google/qsim) simulator. However, qsim has plenty of support for being run in a [Google Cloud instance](/qsim/tutorials/gcp_before_you_begin){:.external}, with a variable amount of compute power. In order to get the most capacity possible for qsim, use [Multinode Simulation](/qsim/tutorials/multinode).

For an example of building and running a much larger circuit, see the [QVM Stabilizer Example](/cirq/simulate/qvm_stabilizer_example) tutorial. 

## **Execute** Your Circuit on the Quantum Virtual Machine

You can run the now device-ready circuit, as you would with any other `cirq.Engine` instance, by getting a sampler from it and using the `run` function on the circuits. Your choice of `repetitions` is intrinsically related to the accuracy of your simulated results. We recommend 3000 repetitions for trial runs, and 10,000 repetitions for accuracy-critical runs, but you can stick to one to ten repetitions when testing a code pipeline. You can read more about this in [this paper](https://arxiv.org/abs/2111.02396){:.external}.

In [ ]:
# @title Execute your device ready circuit on the Quantum Virtual Machine

repetitions = 3000
start = time.time()
results = sim_engine.get_sampler(processor_id).run(translated_circuit, repetitions=repetitions)
elapsed = time.time() - start

print('Circuit successfully executed on your quantum virtual machine', processor_id)
print(f'QVM runtime: {elapsed:.04g}s ({repetitions} repetitions)')
print('You can now print or plot "results"')

## **Visualize** Output

Finally, you can use a [state histogram](/cirq/simulate/state_histograms) to plot the measured results.

In [ ]:
ax = cirq.plot_state_histogram(results.histogram(key='out'))
ax.get_xaxis().set_ticks([])
plt.gcf().set_size_inches(10, 4)
plt.show(ax)

The leftmost and rightmost bars correspond to $|00..0\rangle$ and $|11..1\rangle$, respectively, which matches expectation for a GHZ state. Natural $|1\rangle → |0\rangle$ decay causes $|11..1\rangle$ to be much less common than $|00..0\rangle$, and other states also appear due to the various error mechanisms in the hardware that are mimicked by the simulated noise model. Learn more about these errors [here](https://arxiv.org/abs/2111.02396){:.external}.
